# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. 

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта. 

**В нашем распоряжении 5 таблиц:**
1. Таблица books. Содержит данные о книгах:
    - book_id — идентификатор книги;
    - author_id — идентификатор автора;
    - title — название книги;
    - num_pages — количество страниц;
    - publication_date — дата публикации книги;
    - publisher_id — идентификатор издателя.
2. Таблица authors. Содержит данные об авторах:
    - author_id — идентификатор автора;
    - author — имя автора.
3. Таблица publishers. Содержит данные об издательствах:
    - publisher_id — идентификатор издательства;
    - publisher — название издательства;
4. Таблица ratings. Содержит данные о пользовательских оценках книг:
    - rating_id — идентификатор оценки;
    - book_id — идентификатор книги;
    - username — имя пользователя, оставившего оценку;
    - rating — оценка книги.
5. Таблица reviews. Содержит данные о пользовательских обзорах на книги:
    - review_id — идентификатор обзора;
    - book_id — идентификатор книги;
    - username — имя пользователя, написавшего обзор;
    - text — текст обзора.

In [2]:
# импортируем библиотеки
import pandas as pd
import psycopg2 as ps
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Обзор данных

### Таблица books

Выведем первые строки и посчитаем сколько всего строк в таблице.

In [3]:
query = '''
SELECT * 
FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''
SELECT COUNT(*) 
FROM books'''
get_sql_data(query)

,count
0,1000


В таблице с книгами ровно 1000 записей. Есть колонки с айди автора и издательства, которые, скорее всего, являются внешними ключами и с их помощью мы в дальнейшем будем объединять таблицы. Также в таблице содержится основная информация о книгах: название, количество страниц и дата публикации.

### Таблица authors

Аналогично рассмотрим таблицу с авторами.

In [5]:
query = '''
SELECT * 
FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
SELECT COUNT(*) 
FROM authors'''
get_sql_data(query)

,count
0,636


Здесь 636 записи об авторах. В таблице, как и предполагалось, есть айди автора и его ФИО.

### Таблица publishers

Изучим таблицу с издательствами.

In [7]:
query = '''
SELECT * 
FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query = '''
SELECT COUNT(*) 
FROM publishers'''
get_sql_data(query)

,count
0,340


Всего нам дано 340 издательств. В таблице представлены их айди и название издательства.

### Таблица ratings

Рассмотрим таблицу, содержащую данные о пользовательских оценках книг.

In [9]:
query = '''
SELECT * 
FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query = '''
SELECT COUNT(*) 
FROM ratings'''
get_sql_data(query)

,count
0,6456


У нас в распоряжении 6456 записей с оценками пользователей. В таблице есть айди книги, что поможет связать эту таблицу с первой, а также имя и сама оценка пользователя.

### Таблица reviews

Осталось раасмотреть последнюю таблицу reviews. В ней хранятся обзоры на книги.

In [11]:
query = '''
SELECT * 
FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [12]:
query = '''
SELECT COUNT(*) 
FROM reviews'''
get_sql_data(query)

,count
0,2793


Всего 2793 обзора. Здесь снова есть айди книги, а также уникальное имя пользоватля, эти значения нужны для объединения с таблицами books и ratings. Также в данных есть непосредственно сам текст обзора.

## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [13]:
task1 = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(task1)

,count
0,819


С начала 2000 года по настоящее время вышло 819 книг, имеющихся на платформе. Это 81,9% от общего числа представленных книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [14]:
task2 = '''
SELECT b.book_id,
       b.title,
       AVG(rt.rating) AS avg_rating,
       COUNT(DISTINCT rw.review_id) AS review_count
FROM books AS b
LEFT JOIN ratings AS rt ON rt.book_id=b.book_id
LEFT JOIN reviews AS rw ON rw.book_id=b.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC, review_count DESC
'''
get_sql_data(task2)

,book_id,title,avg_rating,review_count
0,17,A Dirty Job (Grim Reaper #1),5.00,4
1,553,School's Out—Forever (Maximum Ride #2),5.00,3
2,444,Moneyball: The Art of Winning an Unfair Game,5.00,3
3,86,Arrows of the Queen (Heralds of Valdemar #1),5.00,2
4,972,Wherever You Go There You Are: Mindfulness Me...,5.00,2
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,2.25,3
996,202,Drowning Ruth,2.00,3
997,316,His Excellency: George Washington,2.00,2
998,371,Junky,2.00,2


Максимальное количество обзоров на книгу - 4 - у книги "A Dirty Job". На платформе много книг, средний рейтинг которых равен 5.0. Самый низкий рейтинг у книги "Harvesting the Heart", он равен 1.5.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [15]:
task3 = '''
SELECT p.publisher AS publisher,
       COUNT(DISTINCT b.book_id) AS books_count
FROM publishers AS P
LEFT JOIN books AS b ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY books_count DESC
LIMIT 1
'''
get_sql_data(task3)

,publisher,books_count
0,Penguin Books,42


Издательство Penguin Books оказалось лидером по выпуску книг толще 50 страниц. За все время оно выпустило 42 книги.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [16]:
task4 = '''
SELECT a.author AS author,
       AVG(r.rating) AS rating
FROM authors AS a
LEFT JOIN books AS b ON b.author_id=a.author_id
LEFT JOIN ratings AS r ON r.book_id=b.book_id
WHERE b.book_id IN (SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50)
GROUP BY author
ORDER BY rating DESC
LIMIT 1
'''
get_sql_data(task4)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. Средний рейтинг его книги - 4.287.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [25]:
task5 = '''
WITH new AS (SELECT username,
COUNT(rating_id) as count_rating
FROM ratings
GROUP BY username),

helper AS (SELECT COUNT(text) AS cnt_reviews
FROM reviews
FULL OUTER JOIN new ON new.username = reviews.username
WHERE count_rating > 48
GROUP BY reviews.username)

SELECT AVG(cnt_reviews)
FROM helper
'''
get_sql_data(task5)

,avg
0,24.0


В среднем пользователи, которые поставили более 48 оценок, написали 24 отзыва о книгах. 